<a href="https://colab.research.google.com/github/jaydeepthik/NMT-neural-machine-translation/blob/master/NMT_ger2eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount("/content/drive")

In [0]:
import numpy as np
import string
from unicodedata import normalize
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

#read file and make pairs

def read_file(file):
    
    with open(file, 'r', encoding='utf8') as f:
        pairs = [line.strip().split('\t') for line in f]
        #print(pairs)
    return pairs

def preprocess_pairs(data):
    cleaned = list()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    re_print = re.compile('[^%s]' % re.escape(string.printable))    
    
    for pairs in data:
      clean_pair = list()
      for line in pairs:
        line = normalize('NFD', line).encode('ascii', 'ignore')
        line = line.decode('utf8')
        line = line.lower()
        line = line.split()
        line = [re_punc.sub('', w) for w in line]
        line = [re_print.sub('', w) for w in line]
        line = [word for word in line if word.isalpha()]
        #print(line)
        clean_pair.append(" ".join(line))
      cleaned.append(clean_pair)
    return cleaned

def encode_optput(sequences, vocab_size):
  ylist = list()
  for sequence in sequences:
    encode = to_categorical(sequence, vocab_size)
    ylist.append(encode)
  y = np.array(ylist)
  y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
  return y

In [0]:
data = read_file("/content/drive/My Drive/Colab Notebooks/nmt_data/deu-eng/deu.txt")
data = preprocess_pairs(data)
n_sentences = 10000

reduced_data = data[:10000]
reduced_data = np.array(reduced_data)
np.random.shuffle(reduced_data)

train_data, test_data = reduced_data[:9000], reduced_data[9000:]



In [0]:
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(reduced_data[:,0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_max_len = max(len(line.split()) for line in reduced_data[:,0])

ger_tokenizer = Tokenizer()
ger_tokenizer.fit_on_texts(reduced_data[:,1])
ger_vocab_size = len(ger_tokenizer.word_index)+1
ger_max_len = max(len(line.split()) for line in reduced_data[:,1])


X_train = ger_tokenizer.texts_to_sequences(train_data[:,1])
X_train = pad_sequences(X_train, maxlen=ger_max_len, padding='post')

y_train = eng_tokenizer.texts_to_sequences(train_data[:,0])
y_train = pad_sequences(y_train, maxlen=eng_max_len, padding='post')
y_train = encode_optput(y_train, eng_vocab_size)


X_test = ger_tokenizer.texts_to_sequences(test_data[:,1])
print(X_test[0])
X_test = pad_sequences(X_test, maxlen=ger_max_len, padding='post')

y_test = eng_tokenizer.texts_to_sequences(test_data[:,0])
y_test = pad_sequences(y_test, maxlen=eng_max_len, padding='post')
y_test = encode_optput(y_test, eng_vocab_size)




In [53]:
import keras
from keras.models import Sequential
from keras import layers

model = Sequential()
model.add(layers.Embedding(input_dim=ger_vocab_size,output_dim = 256,input_length= ger_max_len, mask_zero=True))
model.add(layers.LSTM(256))
model.add(layers.RepeatVector(eng_max_len))
model.add(layers.LSTM(256, return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(eng_vocab_size, activation='softmax')))

model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['acc'])          
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 10, 256)           912896    
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 5, 256)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 5, 256)            525312    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 5, 2233)           573881    
Total params: 2,537,401
Trainable params: 2,537,401
Non-trainable params: 0
_________________________________________________________________


In [54]:
model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 9000 samples, validate on 1000 samples
Epoch 1/30
9000/9000 [==============================] - 10s 1ms/step - loss: 4.1875 - acc: 0.4353 - val_loss: 3.4689 - val_acc: 0.4496
Epoch 2/30
9000/9000 [==============================] - 6s 632us/step - loss: 3.3012 - acc: 0.4649 - val_loss: 3.3310 - val_acc: 0.4738
Epoch 3/30
9000/9000 [==============================] - 6s 619us/step - loss: 3.1667 - acc: 0.4762 - val_loss: 3.2589 - val_acc: 0.4810
Epoch 4/30
9000/9000 [==============================] - 5s 579us/step - loss: 3.0388 - acc: 0.4911 - val_loss: 3.1434 - val_acc: 0.4956
Epoch 5/30
9000/9000 [==============================] - 5s 573us/step - loss: 2.8658 - acc: 0.5067 - val_loss: 3.0316 - val_acc: 0.5098
Epoch 6/30
9000/9000 [==============================] - 5s 576us/step - loss: 2.7047 - acc: 0.5268 - val_loss: 2.9028 - val_acc: 0.5350
Epoch 7/30
9000/9000 [==============================] - 5s 568us/step - loss: 2.5342 - ac